In [2]:
import tensorflow as tf
tf.enable_eager_execution()
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from datetime import datetime
import re
from datetime import timedelta
from scipy import stats
from dateutil.relativedelta import relativedelta

In [4]:
data=["06/Nov/2020:00:43:04 +0200"]
df=pd.DataFrame(data,columns=['date'])
def MonthToNum(val):
    dictMonth={'Jan':'1','Feb':'2','Mar':'3','Apr':'4','May':'5','Jun':'6','Jul':'7','Aug':'8','Sep':'9','Oct':'10','Nov':'11','Dec':'12'}
    pattern = '|'.join(sorted(re.escape(k) for k in dictMonth))
    #print(pattern)
    retval=re.sub(pattern, lambda m: dictMonth.get(m.group(0)), val, flags=re.IGNORECASE)
    return retval
    
df['date']=df['date'].apply(lambda x: pd.to_datetime(MonthToNum(x[0:11])+" "+x[12:20]) + timedelta(int(x[20:24])))
print(df)
print(type(df['date']))


                 date
0 2020-06-13 00:43:04
<class 'pandas.core.series.Series'>


In [5]:
from datetime import timedelta

time_string = '24:54:03.294820'
hours, minutes, seconds = map(float, time_string.split(':'))
t_delta = timedelta(hours=hours, minutes=minutes, seconds=seconds)
print(t_delta)

1 day, 0:54:03.294820


In [7]:
print({1 if 1==1 else 0 :'a'})

{1: 'a'}


In [8]:
source = {'a': 10, 'b': 20, 'c': 30}
#destination = {'x': None, 'y': None, 'z': None, 'qq': 'Some value'}
mapping = {"y": "a", "z": "b", "x": "c"}
result={k:source.get(v) for (k,v) in mapping.items()}

print(result)

{'y': 10, 'z': 20, 'x': 30}


In [8]:
abc = ["1","2","3","4","5","A"]

def IsNumeric(value):
     try:
        return int(value)
     except:
        return value
    
abc=list(map(IsNumeric,abc))
print(abc)

[1, 2, 3, 4, 5, 'A']


In [9]:
longtext = """2013 this is data 3 (more data from 3)
2016 this is data 6 (more data from 6)
2011 this is data 1 (more data from 1)
2012 this is data 2 (more data from 2)
2014 this is data 4 (more data from 4)"""

data = longtext.splitlines()
dct={}
for x in data:
    dct[(re.search(r'\d{4}',x))[0]]=x
    
print(dct)    
for i in sorted (dct.keys(),reverse=True) :  
    print(dct.get(i))

{'2013': '2013 this is data 3 (more data from 3)', '2016': '2016 this is data 6 (more data from 6)', '2011': '2011 this is data 1 (more data from 1)', '2012': '2012 this is data 2 (more data from 2)', '2014': '2014 this is data 4 (more data from 4)'}
2016 this is data 6 (more data from 6)
2014 this is data 4 (more data from 4)
2013 this is data 3 (more data from 3)
2012 this is data 2 (more data from 2)
2011 this is data 1 (more data from 1)


In [10]:
def get_pvalue ( con_conv, test_conv, con_size, test_size):
    lift= - abs(test_conv - con_conv)
    scale_one = con_conv * (1-con_conv) * (1/con_size)
    scale_two= test_conv * (1-test_conv) * (1/test_size)
    scale_val = (scale_one + scale_two) **0.5
    p_value=2*stats.norm.cdf(lift, loc=0, scale=scale_val)
    return p_value

def get_power(n, p1, p2, cl):
    alpha = 1 - cl
    qu = stats.norm.ppf(1 - alpha/2)
    diff = abs(p2-p1)
    bp = (p1+p2) / 2
    
    v1 = p1 * (1-p1)
    v2 = p2 * (1-p2)
    bv = bp * (1-bp)
    
    power_part_one = stats.norm.cdf((n**0.5 * diff - qu * (2 * bv)**0.5) / (v1+v2) ** 0.5)
    power_part_two = 1 - stats.norm.cdf((n**0.5 * diff + qu * (2 * bv)**0.5) / (v1+v2) ** 0.5)
    
    power = power_part_one + power_part_two
    
    return (power)

con_conv=0.034351
test_conv=0.041984
con_size=48236
test_size=49867

p_value=get_pvalue(con_conv,test_conv,con_size,test_size)
print(p_value)

power= get_power (test_size, con_conv, test_conv, 0.95)
print(power)

4.257297485586909e-10
0.9999925941372282


In [11]:
A = {1 : "one", 2 : "two"}
B = {2 : "dva", 3 : "three"}

z1=zip(A.keys(),A.values())
z2=zip(B.keys(),B.values())

mylist=[]
mylist.append(list(z1))
mylist.append(list(z2))

variables=[]
phrases=[]
for item in mylist:
    for k,v in item:
        print(k,v)
        variables.append(k)
        phrases.append(v)
df=pd.DataFrame(zip(variables,phrases),columns=['variable','phrase'])
print(df)
grpDictionary=df.groupby('variable').groups

resultDictionary={}
for key, values in grpDictionary.items():
    resultDictionary[key]=" ".join(df.iloc[values]['phrase'])

print(resultDictionary)

1 one
2 two
2 dva
3 three
   variable phrase
0         1    one
1         2    two
2         2    dva
3         3  three
{1: 'one', 2: 'two dva', 3: 'three'}


In [12]:
key_list=['a', 'a', 'a', 'a', 'a', 'a', 'b', 'b', 'c', 'c', 'd']
value_list=['(a)one', '(a)two', '(a)three', '(a)four', '(a)five', '(a)six', '(b)one', '(b)two', '(c)one', '(c)two', '(d)one']

data=list(zip(key_list,value_list))
df=pd.DataFrame(data,columns=['variable','phrase'])
unique_variables=list(df.groupby('variable').groups)
phrases=[]
for key in unique_variables:
    filter=df['variable']==key
    phrases.append("".join(df[filter]['phrase']))
    
print(list(zip(unique_variables,phrases)))

[('a', '(a)one(a)two(a)three(a)four(a)five(a)six'), ('b', '(b)one(b)two'), ('c', '(c)one(c)two'), ('d', '(d)one')]


In [13]:
data=['Period1Monetary','Period2Monetary']
current_date=pd.to_datetime('03-01-2020')

mapDict={1:'Period1',2:'Period2'}

columns=[]
for i in range(1,13):
    columns.append(current_date+ relativedelta(months=i-1))

print([x.strftime('%m-%d-%Y') for x in columns])

['03-01-2020', '04-01-2020', '05-01-2020', '06-01-2020', '07-01-2020', '08-01-2020', '09-01-2020', '10-01-2020', '11-01-2020', '12-01-2020', '01-01-2021', '02-01-2021']


In [14]:
data=[('Team1','Player1',       2,             1,                    1),
('Team1','Player2',       3,             1,                    0),
('Team2','Player3',       2,             2,                    1)]

df=pd.DataFrame(data=data,columns=['Team','Player','Goals', 'YellowCards','RedCards'])
print(df)
fp=df.pivot_table(index='Team',aggfunc='sum')
fp['Totals'] = fp.sum(axis='columns')
fp.loc[('Grand Total'), :] = fp.sum()
print(fp)
#

    Team   Player  Goals  YellowCards  RedCards
0  Team1  Player1      2            1         1
1  Team1  Player2      3            1         0
2  Team2  Player3      2            2         1
             Goals  RedCards  YellowCards  Totals
Team                                             
Team1          5.0       1.0          2.0     8.0
Team2          2.0       1.0          2.0     5.0
Grand Total    7.0       2.0          4.0    13.0


In [15]:
import re
data="ABBAABBBAAABABABA"
matches=re.finditer(r"(ABA)+",data)
for match in matches:
    print(match)

<re.Match object; span=(10, 13), match='ABA'>
<re.Match object; span=(14, 17), match='ABA'>


In [16]:
#x = tf.Variable(tf.zeros([2, 2], tf.int32))
x=torch.zeros(2,2)
item = torch.tensor([[1, 2], [3, 4]])
#tf.assign()
print(x.shape)

#x.assign([1,2],[3,4])
print(x)
print('Only Variables support sliced assignment')

a=tf.Variable([[1,2],[3,4]])
a.assign([[5,6], [7,8]])
print(a.numpy())

data_tensor=tf.ragged.constant([[1, 2], [3, 4,5]])
print(data_tensor)

torch.Size([2, 2])
tensor([[0., 0.],
        [0., 0.]])
Only Variables support sliced assignment
[[5 6]
 [7 8]]
<tf.RaggedTensor [[1, 2], [3, 4, 5]]>


In [17]:
a = torch.tensor([[[1., 0., 0., 0.]],
        [[0., 1., 0., 0.]],
        [[1., 0., 0., 0.]],
        [[0., 0., 0., 1.]],
        [[1., 0., 0., 0.]],
        [[0., 0., 0., 1.]],
        [[1., 0., 0., 0.]]])
print(a[2][0][0])

tensor(1.)


In [18]:
df = pd.DataFrame([[1, 2],
                   [1, 0],
                   [1, 1],
                   [2, 1],
                   [2, 2],
                   [3, 0],
                   [3, 2],
                   [4, 1],
                   [4, 2],
                   [np.nan, 0],
                   [np.nan, 1],
                   [np.nan, 0]], columns=['ids', 'dim'])

item=torch.tensor(df.values)
print(item)

tensor([[1., 2.],
        [1., 0.],
        [1., 1.],
        [2., 1.],
        [2., 2.],
        [3., 0.],
        [3., 2.],
        [4., 1.],
        [4., 2.],
        [nan, 0.],
        [nan, 1.],
        [nan, 0.]], dtype=torch.float64)


In [49]:
d={1:[1,5,3],2:[4,2,6],3:[7,1,9]}

df=pd.DataFrame(d).T
df=df.reset_index()
df=df.sort_values(by=1,axis=0)
print(df)


   index  0  1  2
2      3  7  1  9
1      2  4  2  6
0      1  1  5  3


In [ ]:
('Sunny',1,1,'Rich''Cinema'),
('Sunny',0,1)